# Vegetation Health Index (VHI)
---
XXX

In [ ]:
import ee
import geemap

AttributeError: 'method' object has no attribute 'markers' and no __dict__ for setting new attributes

In [2]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [3]:
# ------------------------------------
# User Inputs
# ------------------------------------
aoi = ee.Geometry.Polygon(
[[[5.608534480510099, 46.698303309141494],
[5.608534480510099, 45.80952394852957],
[7.476210261760099, 45.80952394852957],
[7.476210261760099, 46.698303309141494]]])
Map.addLayer(aoi,{},'AOI')
Map.centerObject(aoi, 9)

startDate = '2023-01-01'
endDate = '2024-01-01'

In [4]:
# ------------------------------------
# Load Landsat 8 Collection 2 Level 2
# ------------------------------------
collection = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(aoi)
    .filterDate(startDate, endDate)
    .filter(ee.Filter.lt('CLOUD_COVER', 20))
)

In [5]:
# ------------------------------------
# Functions
# ------------------------------------
def apply_scaling(image):
    # Surface Reflectance scaling
    sr = image.select(['SR_B4', 'SR_B5']) \
              .multiply(0.0000275).add(-0.2)

    # Land Surface Temperature scaling (Kelvin)
    lst = image.select('ST_B10') \
               .multiply(0.00341802).add(149.0)

    return image.addBands(sr, None, True) \
                .addBands(lst.rename('LST'), None, True)

def add_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return image.addBands(ndvi)

In [6]:
# ------------------------------------
# Preprocess collection
# ------------------------------------
processed = (
    collection
    .map(apply_scaling)
    .map(add_ndvi)
)

In [7]:
# ------------------------------------
# Compute NDVI & LST statistics
# ------------------------------------
ndvi_min = processed.select('NDVI').min()
ndvi_max = processed.select('NDVI').max()

lst_min = processed.select('LST').min()
lst_max = processed.select('LST').max()

In [8]:
# ------------------------------------
# Use median composite
# ------------------------------------
image = processed.median()

In [9]:
# ------------------------------------
# Compute VCI
# ------------------------------------
vci = image.expression(
    '100 * (ndvi - ndvi_min) / (ndvi_max - ndvi_min)', {
        'ndvi': image.select('NDVI'),
        'ndvi_min': ndvi_min,
        'ndvi_max': ndvi_max
    }).rename('VCI')

# ------------------------------------
# Compute TCI
# ------------------------------------
tci = image.expression(
    '100 * (lst_max - lst) / (lst_max - lst_min)', {
        'lst': image.select('LST'),
        'lst_min': lst_min,
        'lst_max': lst_max
    }).rename('TCI')

In [10]:
# ------------------------------------
# Compute VHI
# ------------------------------------
vhi = vci.multiply(0.5).add(tci.multiply(0.5)).rename('VHI')

# ------------------------------------
# Final Image
# ------------------------------------
vhi_image = image.addBands([vci, tci, vhi]).clip(aoi)

# Compute min and max values over the AOI
vhi_stats = vhi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': vhi_stats.get('VHI_min').getInfo(),
    'max': vhi_stats.get('VHI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize NDVI
Map.addLayer(vhi.clip(aoi), vis_params, 'Landsat | VHI')

In [ ]:
# ------------------------------------
# Export to Google Drive
# ------------------------------------
task = ee.batch.Export.image.toDrive(
    image=vhi_image.select('VHI'),
    description='Landsat8_VHI',
    folder='GEE',
    scale=30,
    region=roi,
    maxPixels=1e13
)

task.start()

print("Export started!")

## Monthly VHI
Dataset: Landsat 8 Collection 2 Level-2
Cloud masking: QA_PIXEL (cloud, cloud shadow, cirrus, snow)
Temporal resolution: Monthly composites
Baseline: Same period used for NDVI/LST min–max (can easily be expanded to multi-year)
Plot: Mean VHI over ROI vs time

In [21]:
import ee
import geemap

import pandas as pd
import matplotlib.pyplot as plt

In [22]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [31]:
# ------------------------------------
# USER INPUTS
# ------------------------------------
roi = ee.Geometry.Polygon(
[[[5.608534480510099, 46.698303309141494],
[5.608534480510099, 45.80952394852957],
[7.476210261760099, 45.80952394852957],
[7.476210261760099, 46.698303309141494]]])
Map.addLayer(aoi,{},'AOI')
Map.centerObject(aoi, 9)

start_date = '2022-01-01'
end_date = '2023-12-31'

In [32]:
# ------------------------------------
# Cloud Mask using QA_PIXEL
# ------------------------------------
def mask_landsat_qa(image):
    qa = image.select('QA_PIXEL')

    # Bits (Landsat C2 L2)
    cloud_shadow = 1 << 4
    snow = 1 << 5
    cloud = 1 << 3
    cirrus = 1 << 2

    mask = (
        qa.bitwiseAnd(cloud_shadow).eq(0)
        .And(qa.bitwiseAnd(snow).eq(0))
        .And(qa.bitwiseAnd(cloud).eq(0))
        .And(qa.bitwiseAnd(cirrus).eq(0))
    )

    return image.updateMask(mask)

In [33]:
# ------------------------------------
# Scaling Functions
# ------------------------------------
def apply_scaling(image):
    # Surface Reflectance
    sr = image.select(['SR_B4', 'SR_B5']) \
              .multiply(0.0000275).add(-0.2)

    # Land Surface Temperature (Kelvin)
    lst = image.select('ST_B10') \
               .multiply(0.00341802).add(149.0) \
               .rename('LST')

    return image.addBands(sr, None, True).addBands(lst, None, True)

def add_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return image.addBands(ndvi)

In [34]:
# ------------------------------------
# Load & Preprocess Collection
# ------------------------------------
collection = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(roi)
    .filterDate(start_date, end_date)
    .map(mask_landsat_qa)
    .map(apply_scaling)
    .map(add_ndvi)
)

In [35]:
# ------------------------------------
# NDVI & LST Min/Max (Baseline)
# ------------------------------------
ndvi_min = collection.select('NDVI').min()
ndvi_max = collection.select('NDVI').max()
lst_min = collection.select('LST').min()
lst_max = collection.select('LST').max()

In [39]:
# ------------------------------------
# Monthly VHI Function
# ------------------------------------
def monthly_vhi(year, month):
    start = ee.Date.fromYMD(year, month, 1)
    end = start.advance(1, 'month')

    monthly_col = collection.filterDate(start, end)

    # If collection is empty, return empty image
    img = ee.Image(
        ee.Algorithms.If(
            monthly_col.size().gt(0),
            monthly_col.median(),
            ee.Image()
        )
    )

    # Check NDVI band exists
    has_ndvi = img.bandNames().contains('NDVI')

    vhi = ee.Image(
        ee.Algorithms.If(
            has_ndvi,

            # --- Compute VHI ---
            img.expression(
                '0.5 * (100 * (ndvi - ndvi_min) / (ndvi_max - ndvi_min)) + \
                 0.5 * (100 * (lst_max - lst) / (lst_max - lst_min))',
                {
                    'ndvi': img.select('NDVI'),
                    'ndvi_min': ndvi_min,
                    'ndvi_max': ndvi_max,
                    'lst': img.select('LST'),
                    'lst_min': lst_min,
                    'lst_max': lst_max
                }
            ).rename('VHI'),

            # --- Empty image fallback ---
            ee.Image().rename('VHI')
        )
    )

    return (
        vhi
        .set('year', year)
        .set('month', month)
        .set('system:time_start', start.millis())
    )

In [40]:
# ------------------------------------
# Create Monthly ImageCollection
# ------------------------------------
years = ee.List.sequence(2021, 2022)
months = ee.List.sequence(1, 12)

monthly_images = ee.ImageCollection.fromImages(
    years.map(
        lambda y: months.map(
            lambda m: monthly_vhi(ee.Number(y), ee.Number(m))
        )
    ).flatten()
)

In [41]:
# ------------------------------------
# Extract Mean VHI Time Series
# ------------------------------------
def region_mean(image):
    mean = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=30,
        maxPixels=1e13
    )
    return ee.Feature(None, {
        'date': ee.Date(image.get('system:time_start')).format('YYYY-MM'),
        'VHI': mean.get('VHI')
    })

vhi_ts = monthly_images.map(region_mean).getInfo()

In [42]:
# ------------------------------------
# Convert to Pandas DataFrame
# ------------------------------------
dates = []
values = []

for f in vhi_ts['features']:
    if f['properties']['VHI'] is not None:
        dates.append(f['properties']['date'])
        values.append(f['properties']['VHI'])

df = pd.DataFrame({
    'Date': pd.to_datetime(dates),
    'VHI': values
}).sort_values('Date')

# ------------------------------------
# Plot Time Series
# ------------------------------------
plt.figure(figsize=(10, 5))
plt.plot(df['Date'], df['VHI'], marker='o')
plt.axhline(40, linestyle='--', linewidth=1)
plt.axhline(60, linestyle='--', linewidth=1)
plt.ylabel('VHI')
plt.xlabel('Date')
plt.title('Monthly Vegetation Health Index (VHI)')
plt.grid(True)
plt.tight_layout()
plt.show()

KeyError: 'VHI'